In [1]:
import numpy as np
import pandas as pd
import os

folder = os.getcwd() ; print folder

/home/arda/Documents/DSG


In [2]:
#pd_train = pd.io.pickle.read_pickle(folder + '/data_munged/pd_train_tagged')
#pd_test = pd.io.pickle.read_pickle(folder + '/data_munged/pd_test_tagged')

pd_train = pd.io.pickle.read_pickle(folder + '/data_munged/pd_train_munged')
pd_test = pd.io.pickle.read_pickle(folder + '/data_munged/pd_test_munged')

In [31]:
from sklearn.pipeline import make_pipeline, make_union
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score

dico_pattern={'match_lowercase_only':'\\b[a-z]+\\b',
              'match_word':'\\w{1,}',
              'match_word1': '(?u)\\b\\w+\\b',
              'match_word_punct': '\w+|[,.?!;]',
              'match_NNP': '\\b[A-Z][a-z]+\\b|\\b[A-Z]+\\b',
              'match_punct': "[,.?!;'-]"
             }
tfv = TfidfVectorizer(lowercase=False, stop_words=None, token_pattern=dico_pattern["match_word_punct"], 
                      ngram_range=(1, 2), max_df=1.0, min_df=2, max_features=None, 
                      vocabulary=None, binary=True, norm=u'l2', 
                      use_idf=True, smooth_idf=True, sublinear_tf=False)

clf = KMeans(n_clusters=6, init='k-means++', n_init=10, max_iter=300, tol=0.0001, 
             precompute_distances='auto', verbose=0, random_state=None, copy_x=True, n_jobs=1)

pipeline = make_pipeline(ColumnSelector(key='Sentence'), tfv)

In [9]:
Y = pd_train['Author']
X = pipeline.fit_transform(pd_train) ; print X.shape

(28723, 70877)


In [30]:
clf.fit(X)

KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=6, n_init=10,
    n_jobs=1, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=0)

In [32]:
clf.predict(X)

NotFittedError: This KMeans instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.

In [28]:
accuracy_score(y_true, y_pred)

0              twain
1              twain
2              twain
3              twain
4              twain
5              twain
6              twain
7              twain
8              twain
9              twain
10             twain
11             twain
12             twain
13             twain
14             twain
15             twain
16             twain
17             twain
18             twain
19             twain
20             twain
21             twain
22             twain
23             twain
24             twain
25             twain
26             twain
27             twain
28             twain
29             twain
            ...     
28693    shakespeare
28694    shakespeare
28695    shakespeare
28696    shakespeare
28697    shakespeare
28698    shakespeare
28699    shakespeare
28700    shakespeare
28701    shakespeare
28702    shakespeare
28703    shakespeare
28704    shakespeare
28705    shakespeare
28706    shakespeare
28707    shakespeare
28708    shakespeare
28709    shak

In [ ]:
from sklearn.cross_validation import StratifiedKFold, StratifiedShuffleSplit, cross_val_score

for cv in range(3):
    
    skf = StratifiedKFold(Y, n_folds=5, indices=None, shuffle=True, random_state=None)
    scores_skf = cross_val_score(clf, X,Y,scoring='accuracy',cv=skf, n_jobs=-1)

    sss = StratifiedShuffleSplit(Y, 5, test_size=0.2, random_state=0)
    scores_sss = cross_val_score(clf, X,Y,scoring='accuracy',cv=sss, n_jobs=-1)

    print ("SLF: acc: %0.4f, std: %0.4f, SSS: acc: %0.4f, std: %0.4f" %
           (scores_skf.mean(), scores_skf.std(), scores_sss.mean(), scores_sss.std()))